In [2]:
import warnings
warnings.filterwarnings('ignore')

path = "../../kaggle/data/"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

#main module for evaluation
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error,r2_score

def calcMetrics(testActualVal, predictions):
    #regression evaluation measures
    data={"RMSLE":[mean_squared_log_error(testActualVal, predictions)**0.5],
         "MAE":[mean_absolute_error(testActualVal, predictions)],
         "RMSE":[mean_squared_error(testActualVal, predictions)**0.5],
         "R2":[r2_score(testActualVal, predictions)]}
    metric_df=pd.DataFrame(data)
    return metric_df

def split(df, useful_features, target):
    val_start_index=df.shape[0]-len(df["date"][df["date"] >=pd.to_datetime("2017-01-01")])
    train = df[:val_start_index]
    val=df[val_start_index:]
    X_train = train[useful_features]
    y_train = train[target]
    X_valid = val[useful_features]
    y_valid = val[target]
    return X_train, y_train, X_valid, y_valid

def pl(df_compare, actual, pred):
    fig, ax = plt.subplots(figsize=(14,3))
    df = df_compare[df_compare.date >= pd.to_datetime("2017-01-02")]
    g1=sns.lineplot(data=df, x = "date", y = actual, ax = ax)
    g2=sns.lineplot(data=df, x = "date", y = pred, ax = ax)
    g1.set(yscale='log')
    g2.set(yscale='log')
    plt.show()
    plt.close()

In [8]:
train_merged = pd.read_csv(path + "train_merged.csv")
train_merged["date"] = pd.to_datetime(train_merged.date)

In [9]:
train_merged.head(2)

,date,store_nbr,family,onpromotion,sales,city,state,typestores,cluster,typeholiday,locale,locale_name,description,transferred,dcoilwtico,day_of_week,month,year
0,2013-01-01,1,AUTOMOTIVE,0,0.0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,2,1,2013
1,2013-01-01,1,BABY CARE,0,0.0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,2,1,2013
